# Laboratorio de regresión - 4


<table style="width: 70%; margin: 20px auto; border-collapse: collapse; font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; box-shadow: 0 2px 8px rgba(0, 0, 0, 0.2);">
  <thead>
    <tr style="background: linear-gradient(90deg, #0F2027, #203A43, #2C5364); color: #ffffff; text-transform: uppercase;">
      <th style="padding: 12px 20px; border: 1px solid #ddd;">Nombre del Alumno</th>
      <th style="padding: 12px 20px; border: 1px solid #ddd;">Expediente</th>
      <th style="padding: 12px 20px; border: 1px solid #ddd;">Fecha</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td style="padding: 12px 20px; border: 1px solid #ddd; text-align: center;">Christopher Jacob Ahumada Robles</td>
      <td style="padding: 12px 20px; border: 1px solid #ddd; text-align: center;">748077</td>
      <td style="padding: 12px 20px; border: 1px solid #ddd; text-align: center;">13/02/2025</td>
    </tr>
  </tbody>
</table>
 |   |

## Modelos penalizados

Hasta ahora la función de costo que usamos para decidir qué tan bueno es nuestro modelo al momento de ajustar es:

$$ \text{RSS} = \sum_{i=1}^n e_i^2 = \sum_{i=1}^n (y_i - \hat{y_i})^2 $$

Dado que los errores obtenidos son una combinación de sesgo y varianza, puede ser que se sesgue un parámetro para minimizar el error. Esto significa que el modelo puede decidir que la salida no sea una combinación de los factores, sino una fuerte predilección sobre uno de los factores solamente. 

E.g. se quiere ajustar un modelo

$$ \hat{z} = \hat{\beta_0} + \hat{\beta_1} x + \hat{\beta_2} y $$

Se ajusta el modelo y se decide que la mejor decisión es $\hat{\beta_1} = 10000$ y $\hat{\beta_2}=50$. Considera limitaciones de problemas reales:
- Quizás los parámetros son ajustes de maquinaria que se deben realizar para conseguir el mejor producto posible, y que $10000$ sea imposible de asignar.
- Quizás los datos actuales están sesgados y sólo hacen parecer que uno de los factores importa más que el otro.

Una de las formas en las que se puede mitigar este problema es penalizando a los parámetros del modelo, cambiando la función de costo:

$$ \text{RSS}_{L2} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p \hat{\beta_j}^2 $$

El *L2* significa que se está agregando una penalización de segundo orden. Lo que hace esta penalización es que los factores ahora sólo tendrán permitido crecer si hay una reducción al menos proporcional en el error (sacrificamos sesgo, pero reducimos la varianza).

Asimismo, existe la penalización *L1*

$$ \text{RSS}_{L1} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p |\hat{\beta_j}| $$

A las penalizaciones *L2* y *L1* se les conoce también como Ridge y Lasso, respectivamente.

Para realizar una regresión con penalización de Ridge o de Lasso usamos el objeto `Ridge(alpha=?)` o `Lasso(alpha=?)` en lugar de `LinearRegression()` de `sklearn`.

Utiliza el dataset de publicidad (Advertising.csv) y realiza 3 regresiones múltiples:

$$ \text{sales} = \beta_0 + \beta_1 (\text{TV}) + \beta_2 (\text{radio}) + \beta_3 (\text{newspaper}) + \epsilon $$

1. Sin penalización
2. Con penalización L2
3. Con penalización L1

Compara los resultados de los parámetros y sus *p-values*, y los $R^2$ resultantes.

## Importamos librerias 

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [24]:
import matplotlib.pyplot as plt
%matplotlib inline 

## Obtenemos muestra con la que vamos a trabajar

In [26]:
df=pd.read_csv("Advertising.csv")
df=df.set_index("Unnamed: 0")  #Hacemos indice a columna innecesaria
df.info(), df.describe() #Exploramos las caracteristicas 

<class 'pandas.core.frame.DataFrame'>
Index: 200 entries, 1 to 200
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   TV         200 non-null    float64
 1   radio      200 non-null    float64
 2   newspaper  200 non-null    float64
 3   sales      200 non-null    float64
dtypes: float64(4)
memory usage: 7.8 KB


(None,
                TV       radio   newspaper       sales
 count  200.000000  200.000000  200.000000  200.000000
 mean   147.042500   23.264000   30.554000   14.022500
 std     85.854236   14.846809   21.778621    5.217457
 min      0.700000    0.000000    0.300000    1.600000
 25%     74.375000    9.975000   12.750000   10.375000
 50%    149.750000   22.900000   25.750000   12.900000
 75%    218.825000   36.525000   45.100000   17.400000
 max    296.400000   49.600000  114.000000   27.000000)

## Definimos nuestras variables y creamos la matriz X

In [31]:
# Definir X (variables independientes) y y (variable dependiente)
x_TV=np.reshape(df["TV"],[-1,1])
x_R=np.reshape(df["radio"],[-1,1])
x_N=np.reshape(df["newspaper"],[-1,1])
y = df["sales"].values.reshape(-1, 1)
# Columna de unos para b0
ones=np.ones([len(y),1])
# Matriz X
X=np.hstack((ones,x_TV,x_R,x_N))

## Escalamos los datos con min-max para poder tener comparaciones e insigths no solo 1:1 si no entre caracteristicas de X

In [34]:
escala = MinMaxScaler()
X_esc = escala.fit_transform(X)

## Regresion lineal multiple sin penalizacion

In [37]:
model_ols = sm.OLS(y, X_esc).fit()
print(model_ols.summary())
RSS_lr = np.sum((y - model_ols.fittedvalues) ** 2)
r2_lr = model_ols.rsquared
print(f"RSS de Regresión Lineal: {RSS_lr:.4f}")
print(f"R² de Regresión Lineal: {r2_lr:.4f}")

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.982
Model:                            OLS   Adj. R-squared (uncentered):              0.981
Method:                 Least Squares   F-statistic:                              3533.
Date:                Thu, 13 Feb 2025   Prob (F-statistic):                   5.96e-171
Time:                        10:19:59   Log-Likelihood:                         -424.45
No. Observations:                 200   AIC:                                      854.9
Df Residuals:                     197   BIC:                                      864.8
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

C:\Users\chris\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1967: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])


In [43]:
ridge = Ridge(alpha=1.0)
ridge.fit(X_esc, y)

y_pred_ridge = ridge.predict(X_esc)
RSS_ridge = np.sum((y - y_pred_ridge) ** 2)
r2_ridge = r2_score(y, y_pred_ridge)
print(f"RSS de Ridge: {RSS_ridge:.4f}")
print(f"R² de Ridge: {r2_ridge:.4f}")

RSS de Ridge: 570.9500
R² de Ridge: 0.8946


In [47]:
lasso = Lasso(alpha=0.1)
lasso.fit(X_esc, y)

y_pred_lasso = lasso.predict(X_esc)
RSS_lasso = np.sum((y - y_pred_lasso) ** 2)
r2_lasso = r2_score(y, y_pred_lasso)
print(f"RSS de Lasso: {RSS_lasso:.4f}")
print(f"R² de Lasso: {r2_lasso:.4f}")

RSS de Lasso: 1881417.9600
R² de Lasso: 0.8891


In [51]:
coef_df = pd.DataFrame({
    'Feature': ['Intercept', 'TV', 'Radio', 'Newspaper'],
    'Linear Regression': model.coef_[0],
    'Ridge': ridge.coef_[0],
    'Lasso': lasso.coef_
})
coef_df



AttributeError: 'OLSResults' object has no attribute 'coef_'

In [161]:
# Crear un DataFrame para visualizar los coeficientes y p-values
coef_df = pd.DataFrame({
    'Feature': ['Intercept', 'TV', 'Radio', 'Newspaper'],
    'Linear Regression Coef': model_ols.params,
    'Linear Regression p-value': model_ols.pvalues,
    'Ridge': ridge.coef_[0],
    'Lasso': lasso.coef_
})
coef_df

,Feature,Linear Regression Coef,Linear Regression p-value,Ridge,Lasso
0,Intercept,0.000000,NaN,0.000000,0.000000
1,TV,15.938339,3.100283e-97,12.787960,12.397460
2,Radio,11.061786,2.312767e-59,8.821560,8.263014
3,Newspaper,1.913924,1.308511e-02,0.210209,0.000000


In [169]:
accuracy_df = pd.DataFrame({
    'Model': ['Linear Regression', 'Ridge', 'Lasso'],
    'RSS': [RSS_lr, RSS_ridge, RSS_lasso],
    'R²': [r2_lr, r2_ridge, r2_lasso]
})
accuracy_df

,Model,RSS,R²
0,Linear Regression,2.493777e+06,0.981753
1,Ridge,5.709500e+02,0.894603
2,Lasso,1.881418e+06,0.889095
